In [1]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import chain

In [2]:
from langchain_ollama import ChatOllama, OllamaEmbeddings

In [3]:
# 您是一位非常聪明的物理教授。你善于用简明易懂的方式回答物理问题。当你不知道问题的答案时，你会承认自己不知道。
physics_template = """You are a very smart physics professor. You are great at answering questions about physics in a concise and easy-to-understand manner. When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{query}"""

In [4]:
# 您是一位出色的数学家。你很擅长回答数学问题。你之所以如此出色，是因为你能够将难题分解成各个部分，回答各个部分的问题，然后将它们组合起来回答更广泛的问题。
math_template = """You are a very good mathematician. You are great at answering math questions. You are so good because you are able to break down hard problems into their component parts, answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

In [5]:
embeddings = OllamaEmbeddings(
    base_url="http://localhost:11434",
    model="nomic-embed-text"
)
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

In [6]:
len(prompt_embeddings)

2

In [7]:
# Route question to prompt
@chain
def prompt_router(query):
    # Embed question
    query_embedding = embeddings.embed_query(query)
    # Compute similarity
    # first and second are for physics and math respectively
    similarities = cosine_similarity([query_embedding], prompt_embeddings)
    print(similarities)
    similarity = similarities[0]
    # Pick the prompt most similar to the input question
    most_similar = prompt_templates[similarity.argmax()]
    return PromptTemplate.from_template(most_similar)

In [8]:
llm = ChatOllama(base_url="http://localhost:11434", model="qwen2.5:32b", temperature=0)

semantic_router = prompt_router | llm | StrOutputParser()

In [ ]:
print(semantic_router.invoke("What's a black hole"))

[[0.40115071 0.36440887]]
A black hole is an extremely dense region of space where gravity pulls so much that even light cannot escape from it. This intense gravitational field is created when a massive star collapses in on itself at the end of its life cycle, forming what we call a singularity—a point with infinite density and zero volume.

Black holes are invisible to direct observation because no light can escape them, but their presence can be inferred by observing their effects on nearby matter. For example, as gas and dust fall into a black hole, they heat up and emit X-rays that can be detected by telescopes. Black holes come in different sizes, from stellar black holes (which are about 20 to 100 times more massive than our Sun) to supermassive black holes found at the centers of most galaxies, which can have masses billions of times greater than the Sun.


In [10]:
print(semantic_router.invoke("What's the velocity of light?"))

[[0.45155714 0.35444386]]
The speed of light in a vacuum is approximately \(299,792,458\) meters per second (m/s). This value is often rounded to about \(3 \times 10^8\) m/s for simpler calculations. It's important to note that the term "velocity" typically implies both speed and direction; however, when discussing the speed of light without a specified direction, we generally refer to it as its speed.


In [11]:
print(semantic_router.invoke("Why's Newton's third law of motion?"))

[[0.48621521 0.44135824]]
Newton's Third Law of Motion states that for every action, there is an equal and opposite reaction. This means if object A exerts a force on object B, then object B will exert an equal but oppositely directed force back onto object A.

To put it simply: whenever one object pushes or pulls another, the second object pushes or pulls back just as hard in the opposite direction. This law explains why rockets can launch into space (the rocket pushes down on the exhaust gases, and they push up on the rocket) and why you move forward when you walk (your foot pushes backward against the ground, and the ground pushes your foot forward).
